In [ ]:
!pip install pandas numpy scikit-learn joblib requests

In [3]:
"""
=============================================================================
MULTI-LOCATION COASTAL HAZARD DETECTION SYSTEM - FIXED VERSION
=============================================================================
Complete system for real-time tsunami, cyclone, high wave, and flood detection
across multiple coastal locations worldwide.

Version: 2.0 (Fixed)
Date: November 2024
=============================================================================
"""

# ============================================================================
# IMPORTS
# ============================================================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib
from datetime import datetime, timedelta
from typing import Dict, List, Optional
import json
from collections import defaultdict
import requests
import time
import threading
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION - ADD YOUR API KEY HERE
# ============================================================================

class Config:
    """
    System configuration
    👇 ADD YOUR WEATHERAPI KEY HERE
    """
    # Get your free API key from: https://www.weatherapi.com/
    WEATHER_API_KEY = "3c29688add064b48a1b70521252011"  # ← REPLACE THIS!
    
    # USGS Earthquake API (no key required)
    USGS_API_URL = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.geojson"
    
    # Monitoring intervals
    USGS_POLL_INTERVAL = 180  # seconds (3 minutes)
    WEATHER_POLL_INTERVAL = 300  # seconds (5 minutes)
    
    # Detection thresholds
    MIN_EARTHQUAKE_MAGNITUDE = 5.5
    TSUNAMI_MIN_MAGNITUDE = 6.5
    TSUNAMI_MAX_DEPTH = 70  # km

# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def safe_get_value(data: Dict, key: str, default=0):
    """
    Safely get value from dict, handling None
    This prevents TypeError when API returns None values
    """
    value = data.get(key, default)
    return default if value is None else value

# ============================================================================
# 1. COASTAL LOCATIONS DATABASE
# ============================================================================

class CoastalLocationsDB:
    """
    Database of coastal monitoring locations
    """
    
    LOCATIONS = {
        # India - West Coast
        'mumbai': {
            'name': 'Mumbai',
            'country': 'India',
            'coords': {'lat': 19.0760, 'lon': 72.8777},
            'region': 'Arabian Sea',
            'risk_profile': 'high',
            'population': 12400000,
            'coastline_type': 'west_facing'
        },
        'goa': {
            'name': 'Goa',
            'country': 'India',
            'coords': {'lat': 15.2993, 'lon': 74.1240},
            'region': 'Arabian Sea',
            'risk_profile': 'medium',
            'population': 1458545,
            'coastline_type': 'west_facing'
        },
        'kerala_coast': {
            'name': 'Kerala Coast',
            'country': 'India',
            'coords': {'lat': 10.8505, 'lon': 76.2711},
            'region': 'Arabian Sea',
            'risk_profile': 'medium',
            'population': 33387677,
            'coastline_type': 'west_facing'
        },
        
        # India - East Coast
        'chennai': {
            'name': 'Chennai',
            'country': 'India',
            'coords': {'lat': 13.0827, 'lon': 80.2707},
            'region': 'Bay of Bengal',
            'risk_profile': 'high',
            'population': 7088000,
            'coastline_type': 'east_facing'
        },
        'visakhapatnam': {
            'name': 'Visakhapatnam',
            'country': 'India',
            'coords': {'lat': 17.6868, 'lon': 83.2185},
            'region': 'Bay of Bengal',
            'risk_profile': 'high',
            'population': 2035922,
            'coastline_type': 'east_facing'
        },
        'kolkata': {
            'name': 'Kolkata',
            'country': 'India',
            'coords': {'lat': 22.5726, 'lon': 88.3639},
            'region': 'Bay of Bengal',
            'risk_profile': 'high',
            'population': 14681589,
            'coastline_type': 'east_facing'
        },
        
        # Andaman & Nicobar Islands
        'port_blair': {
            'name': 'Port Blair',
            'country': 'India',
            'coords': {'lat': 11.6234, 'lon': 92.7265},
            'region': 'Andaman Sea',
            'risk_profile': 'critical',
            'population': 100608,
            'coastline_type': 'island'
        },
        
        # Southeast Asia
        'phuket': {
            'name': 'Phuket',
            'country': 'Thailand',
            'coords': {'lat': 7.8804, 'lon': 98.3923},
            'region': 'Andaman Sea',
            'risk_profile': 'critical',
            'population': 416582,
            'coastline_type': 'west_facing'
        },
        'banda_aceh': {
            'name': 'Banda Aceh',
            'country': 'Indonesia',
            'coords': {'lat': 5.5483, 'lon': 95.3238},
            'region': 'Indian Ocean',
            'risk_profile': 'critical',
            'population': 223446,
            'coastline_type': 'island'
        },
        'jakarta': {
            'name': 'Jakarta',
            'country': 'Indonesia',
            'coords': {'lat': -6.2088, 'lon': 106.8456},
            'region': 'Java Sea',
            'risk_profile': 'high',
            'population': 10562088,
            'coastline_type': 'north_facing'
        },
        
        # Pacific
        'tokyo': {
            'name': 'Tokyo Bay',
            'country': 'Japan',
            'coords': {'lat': 35.6762, 'lon': 139.6503},
            'region': 'Pacific Ocean',
            'risk_profile': 'critical',
            'population': 13960000,
            'coastline_type': 'east_facing'
        },
        'manila': {
            'name': 'Manila',
            'country': 'Philippines',
            'coords': {'lat': 14.5995, 'lon': 120.9842},
            'region': 'South China Sea',
            'risk_profile': 'high',
            'population': 1780000,
            'coastline_type': 'west_facing'
        },
    }
    
    @classmethod
    def get_all_locations(cls) -> Dict:
        """Get all monitoring locations"""
        return cls.LOCATIONS
    
    @classmethod
    def get_location(cls, location_id: str) -> Optional[Dict]:
        """Get specific location details"""
        return cls.LOCATIONS.get(location_id)
    
    @classmethod
    def get_high_risk_locations(cls) -> Dict:
        """Get only high-risk and critical locations"""
        return {
            loc_id: loc_data 
            for loc_id, loc_data in cls.LOCATIONS.items() 
            if loc_data['risk_profile'] in ['high', 'critical']
        }
    
    @classmethod
    def get_locations_by_region(cls, region: str) -> Dict:
        """Get all locations in a specific region"""
        return {
            loc_id: loc_data 
            for loc_id, loc_data in cls.LOCATIONS.items() 
            if loc_data['region'] == region
        }

# ============================================================================
# 2. FEATURE ENGINEERING - FIXED
# ============================================================================

class HazardFeatureEngineer:
    """
    Creates features from raw earthquake and weather data
    FIXED: Handles None values from API
    """
    
    @staticmethod
    def create_tsunami_features(earthquake_data, weather_data):
        """
        Create features for tsunami detection
        """
        features = {}
        
        # Seismic features
        features['magnitude'] = earthquake_data['magnitude']
        features['depth'] = earthquake_data['depth']
        features['is_oceanic'] = 1 if earthquake_data['location_type'] == 'oceanic' else 0
        
        # Energy release (derived feature)
        features['energy_release'] = 10 ** (1.5 * earthquake_data['magnitude'] + 4.8)
        
        # Depth category
        if earthquake_data['depth'] < 30:
            features['depth_category'] = 0  # shallow
        elif earthquake_data['depth'] < 70:
            features['depth_category'] = 1  # medium
        else:
            features['depth_category'] = 2  # deep
        
        # Marine features - FIXED: Handle None values
        features['pressure_mb'] = safe_get_value(weather_data, 'pressure_mb', 1013)
        features['wind_kph'] = safe_get_value(weather_data, 'wind_kph', 0)
        features['sig_ht_mt'] = safe_get_value(weather_data, 'sig_ht_mt', 1.0)
        features['swell_period_secs'] = safe_get_value(weather_data, 'swell_period_secs', 10)
        features['tide_height_mt'] = safe_get_value(weather_data, 'tide_height_mt', 2.0)
        features['water_temp_c'] = safe_get_value(weather_data, 'water_temp_c', 25)
        
        # Derived marine features
        features['wave_energy'] = features['sig_ht_mt'] ** 2 * features['swell_period_secs']
        features['pressure_anomaly'] = 1013 - features['pressure_mb']
        
        # Interaction features
        features['mag_depth_ratio'] = features['magnitude'] / (features['depth'] + 1)
        features['mag_wave_interaction'] = features['magnitude'] * features['sig_ht_mt']
        
        return features
    
    @staticmethod
    def calculate_weather_score(weather_data):
        """
        Calculate weather score (0-100) based on hazard indicators
        FIXED: Handle None values
        """
        score = 0
        
        # Wind contribution (0-40 points)
        wind_kph = safe_get_value(weather_data, 'wind_kph', 0)
        if wind_kph < 25:
            score += 0
        elif wind_kph < 40:
            score += 10
        elif wind_kph < 60:
            score += 20
        elif wind_kph < 90:
            score += 30
        else:
            score += 40
        
        # Pressure contribution (0-25 points)
        pressure_mb = safe_get_value(weather_data, 'pressure_mb', 1013)
        if pressure_mb >= 1000:
            score += 0
        elif pressure_mb >= 990:
            score += 10
        elif pressure_mb >= 980:
            score += 20
        else:
            score += 25
        
        # Wave height contribution (0-20 points)
        sig_ht_mt = safe_get_value(weather_data, 'sig_ht_mt', 1.0)
        if sig_ht_mt < 1.5:
            score += 0
        elif sig_ht_mt < 2.5:
            score += 5
        elif sig_ht_mt < 4.0:
            score += 12
        else:
            score += 20
        
        # Precipitation contribution (0-10 points)
        precip_mm = safe_get_value(weather_data, 'precip_mm', 0)
        if precip_mm < 10:
            score += 0
        elif precip_mm < 20:
            score += 5
        else:
            score += 10
        
        # Cloud cover contribution (0-5 points)
        cloud = safe_get_value(weather_data, 'cloud', 0)
        if cloud > 80:
            score += 5
        
        return min(score, 100)

# ============================================================================
# 3. SYNTHETIC DATA GENERATION (For Training)
# ============================================================================

def generate_training_data(n_samples=1000):
    """
    Generate synthetic training data for model development
    In production, replace this with real historical data
    """
    np.random.seed(42)
    
    data = []
    
    for _ in range(n_samples):
        # Generate earthquake parameters
        magnitude = np.random.uniform(5.0, 9.0)
        depth = np.random.uniform(5, 150)
        is_oceanic = np.random.choice([0, 1], p=[0.3, 0.7])
        
        # Generate weather parameters
        pressure_mb = np.random.uniform(950, 1020)
        wind_kph = np.random.uniform(0, 150)
        sig_ht_mt = np.random.uniform(0.5, 8.0)
        swell_period_secs = np.random.uniform(6, 25)
        tide_height_mt = np.random.uniform(0.5, 5.0)
        water_temp_c = np.random.uniform(15, 32)
        precip_mm = np.random.uniform(0, 50)
        cloud = np.random.uniform(0, 100)
        
        # Create features
        earthquake_data = {
            'magnitude': magnitude,
            'depth': depth,
            'latitude': np.random.uniform(-60, 60),
            'longitude': np.random.uniform(-180, 180),
            'location_type': 'oceanic' if is_oceanic else 'land'
        }
        
        weather_data = {
            'pressure_mb': pressure_mb,
            'wind_kph': wind_kph,
            'sig_ht_mt': sig_ht_mt,
            'swell_period_secs': swell_period_secs,
            'tide_height_mt': tide_height_mt,
            'water_temp_c': water_temp_c,
            'precip_mm': precip_mm,
            'cloud': cloud
        }
        
        features = HazardFeatureEngineer.create_tsunami_features(
            earthquake_data, weather_data
        )
        weather_score = HazardFeatureEngineer.calculate_weather_score(weather_data)
        features['weather_score'] = weather_score
        
        # Generate label based on realistic rules
        tsunami_probability = 0
        
        if is_oceanic and magnitude > 6.5 and depth < 70:
            tsunami_probability = 0.7 + np.random.uniform(0, 0.3)
        elif is_oceanic and magnitude > 7.0 and depth < 100:
            tsunami_probability = 0.5 + np.random.uniform(0, 0.4)
        elif is_oceanic and magnitude > 6.0 and depth < 50:
            tsunami_probability = 0.3 + np.random.uniform(0, 0.3)
        else:
            tsunami_probability = np.random.uniform(0, 0.3)
        
        # Add noise
        tsunami_probability = np.clip(tsunami_probability + np.random.normal(0, 0.1), 0, 1)
        
        # Binary label
        features['tsunami_label'] = 1 if tsunami_probability > 0.5 else 0
        features['tsunami_probability'] = tsunami_probability
        
        data.append(features)
    
    return pd.DataFrame(data)

# ============================================================================
# 4. TSUNAMI DETECTION MODEL
# ============================================================================

class TsunamiDetectionModel:
    """
    Machine Learning model for tsunami detection
    """
    
    def __init__(self):
        self.model = None
        self.scaler = StandardScaler()
        self.feature_columns = None
        
    def train(self, df):
        """Train the tsunami detection model"""
        # Prepare features and labels
        feature_cols = [col for col in df.columns if col not in ['tsunami_label', 'tsunami_probability']]
        self.feature_columns = feature_cols
        
        X = df[feature_cols]
        y = df['tsunami_label']
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        # Scale features
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        # Train model
        self.model = GradientBoostingClassifier(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=5,
            random_state=42
        )
        
        self.model.fit(X_train_scaled, y_train)
        
        # Evaluate
        y_pred = self.model.predict(X_test_scaled)
        
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'classification_report': classification_report(y_test, y_pred),
            'confusion_matrix': confusion_matrix(y_test, y_pred),
            'feature_importance': dict(zip(feature_cols, self.model.feature_importances_))
        }
        
        print("=== TSUNAMI MODEL TRAINING RESULTS ===")
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print("\nClassification Report:")
        print(metrics['classification_report'])
        print("\nTop 5 Important Features:")
        sorted_features = sorted(metrics['feature_importance'].items(), 
                                key=lambda x: x[1], reverse=True)
        for feat, imp in sorted_features[:5]:
            print(f"  {feat}: {imp:.4f}")
        
        return metrics
    
    def predict(self, features_dict):
        """
        Predict tsunami probability
        
        ** OUTPUT FOR FRONTEND **
        """
        # Convert to DataFrame
        features_df = pd.DataFrame([features_dict])
        
        # Ensure all required columns exist
        for col in self.feature_columns:
            if col not in features_df.columns:
                features_df[col] = 0
        
        features_df = features_df[self.feature_columns]
        
        # Scale and predict
        features_scaled = self.scaler.transform(features_df)
        probability = self.model.predict_proba(features_scaled)[0][1]
        prediction = self.model.predict(features_scaled)[0]
        
        # Classify risk level
        if probability > 0.7:
            risk_level = "HIGH"
            alert_level = 5
        elif probability > 0.5:
            risk_level = "MEDIUM-HIGH"
            alert_level = 4
        elif probability > 0.3:
            risk_level = "MEDIUM"
            alert_level = 3
        else:
            risk_level = "LOW"
            alert_level = 2
        
        output = {
            'hazard_type': 'TSUNAMI',
            'probability': round(float(probability), 3),
            'risk_level': risk_level,
            'alert_level': alert_level,
            'prediction': 'TSUNAMI LIKELY' if prediction == 1 else 'NO TSUNAMI',
            'confidence': round(float(max(probability, 1-probability)), 3),
            'timestamp': datetime.now().isoformat()
        }
        
        return output
    
    def save_model(self, filepath='tsunami_model.pkl'):
        """Save trained model"""
        joblib.dump({
            'model': self.model,
            'scaler': self.scaler,
            'feature_columns': self.feature_columns
        }, filepath)
        print(f"Model saved to {filepath}")
    
    def load_model(self, filepath='tsunami_model.pkl'):
        """Load trained model"""
        loaded = joblib.load(filepath)
        self.model = loaded['model']
        self.scaler = loaded['scaler']
        self.feature_columns = loaded['feature_columns']
        print(f"Model loaded from {filepath}")

# ============================================================================
# 5. RULE-BASED HAZARD DETECTORS - FIXED
# ============================================================================

class CycloneDetector:
    """
    Rule-based cyclone detection
    FIXED: Handles None values
    """
    
    @staticmethod
    def detect(weather_data):
        """
        Detect cyclone conditions
        
        ** OUTPUT FOR FRONTEND **
        """
        wind_kph = safe_get_value(weather_data, 'wind_kph', 0)
        gust_kph = safe_get_value(weather_data, 'gust_kph', 0)
        pressure_mb = safe_get_value(weather_data, 'pressure_mb', 1013)
        cloud = safe_get_value(weather_data, 'cloud', 0)
        precip_mm = safe_get_value(weather_data, 'precip_mm', 0)
        
        cyclone_score = 0
        category = "NORMAL"
        risk_level = "LOW"
        alert_level = 1
        
        max_wind = max(wind_kph, gust_kph)
        
        if max_wind > 60 and pressure_mb < 995 and cloud > 80:
            cyclone_score += 40
            category = "POSSIBLE STORM"
            risk_level = "MEDIUM"
            alert_level = 3
        
        if max_wind > 90 and pressure_mb < 980:
            cyclone_score += 60
            category = "CYCLONE LIKELY"
            risk_level = "HIGH"
            alert_level = 4
        
        if max_wind > 120 and pressure_mb < 970:
            cyclone_score = 100
            category = "SEVERE CYCLONE"
            risk_level = "CRITICAL"
            alert_level = 5
        
        wind_category = "NORMAL"
        if max_wind > 150:
            wind_category = "SUPER CYCLONE"
        elif max_wind > 120:
            wind_category = "SEVERE CYCLONE"
        elif max_wind > 90:
            wind_category = "CYCLONE"
        elif max_wind > 75:
            wind_category = "STRONG STORM"
        elif max_wind > 60:
            wind_category = "STORM"
        
        output = {
            'hazard_type': 'CYCLONE',
            'detected': cyclone_score > 40,
            'cyclone_score': cyclone_score,
            'category': category,
            'wind_category': wind_category,
            'risk_level': risk_level,
            'alert_level': alert_level,
            'wind_speed_kph': max_wind,
            'pressure_mb': pressure_mb,
            'rainfall_mm': precip_mm,
            'timestamp': datetime.now().isoformat()
        }
        
        return output


class HighWaveDetector:
    """
    Rule-based high wave detection
    FIXED: Handles None values
    """
    
    @staticmethod
    def detect(weather_data):
        """
        Detect high wave conditions
        
        ** OUTPUT FOR FRONTEND **
        """
        sig_ht_mt = safe_get_value(weather_data, 'sig_ht_mt', 1.0)
        swell_period_secs = safe_get_value(weather_data, 'swell_period_secs', 10)
        swell_ht_mt = safe_get_value(weather_data, 'swell_ht_mt', 1.0)
        
        wave_condition = "NORMAL"
        beach_flag = "GREEN"
        alert_level = 1
        risk_level = "LOW"
        
        if sig_ht_mt > 6:
            wave_condition = "EXTREMELY DANGEROUS"
            beach_flag = "BLACK"
            alert_level = 5
            risk_level = "CRITICAL"
        elif sig_ht_mt > 4:
            wave_condition = "DANGEROUS"
            beach_flag = "RED"
            alert_level = 4
            risk_level = "HIGH"
        elif sig_ht_mt > 3:
            wave_condition = "VERY ROUGH"
            beach_flag = "RED"
            alert_level = 3
            risk_level = "MEDIUM"
        elif sig_ht_mt > 2:
            wave_condition = "ROUGH"
            beach_flag = "YELLOW"
            alert_level = 2
            risk_level = "LOW"
        
        rip_current_risk = "LOW"
        if swell_period_secs > 14 and sig_ht_mt > 2.5:
            rip_current_risk = "HIGH"
        elif swell_period_secs > 12 and sig_ht_mt > 1.5:
            rip_current_risk = "MEDIUM"
        
        swell_strength = "NORMAL"
        if swell_period_secs > 20:
            swell_strength = "DANGEROUS SURF"
        elif swell_period_secs > 16:
            swell_strength = "VERY STRONG"
        elif swell_period_secs > 12:
            swell_strength = "STRONG"
        
        output = {
            'hazard_type': 'HIGH_WAVES',
            'detected': sig_ht_mt > 2.5,
            'wave_condition': wave_condition,
            'beach_flag': beach_flag,
            'risk_level': risk_level,
            'alert_level': alert_level,
            'significant_wave_height_m': sig_ht_mt,
            'swell_period_secs': swell_period_secs,
            'swell_strength': swell_strength,
            'rip_current_risk': rip_current_risk,
            'safe_for_swimming': sig_ht_mt < 2.0,
            'timestamp': datetime.now().isoformat()
        }
        
        return output


class FloodDetector:
    """
    Rule-based coastal flood detection
    FIXED: Handles None values
    """
    
    @staticmethod
    def detect(weather_data):
        """
        Detect coastal flood risk
        
        ** OUTPUT FOR FRONTEND **
        """
        tide_height_mt = safe_get_value(weather_data, 'tide_height_mt', 2.0)
        pressure_mb = safe_get_value(weather_data, 'pressure_mb', 1013)
        wind_kph = safe_get_value(weather_data, 'wind_kph', 0)
        wind_dir = weather_data.get('wind_dir', 'N')
        if wind_dir is None:
            wind_dir = 'N'
        precip_mm = safe_get_value(weather_data, 'precip_mm', 0)
        
        flood_score = 0
        risk_level = "LOW"
        alert_level = 1
        
        tide_status = "NORMAL"
        if tide_height_mt > 4.5:
            tide_status = "STORM SURGE RISK"
            flood_score += 50
            alert_level = 5
            risk_level = "CRITICAL"
        elif tide_height_mt > 3.5:
            tide_status = "FLOODING POSSIBLE"
            flood_score += 40
            alert_level = 4
            risk_level = "HIGH"
        elif tide_height_mt > 2.5:
            tide_status = "VERY HIGH TIDE"
            flood_score += 20
            alert_level = 3
            risk_level = "MEDIUM"
        elif tide_height_mt > 1.5:
            tide_status = "HIGH TIDE"
            flood_score += 10
        
        storm_surge_risk = False
        if pressure_mb < 990 and tide_height_mt > 3.0:
            storm_surge_risk = True
            flood_score += 30
            alert_level = max(alert_level, 4)
            risk_level = "HIGH"
        
        onshore_winds = wind_dir in ['E', 'SE', 'NE', 'ESE', 'ENE']
        if onshore_winds and wind_kph > 40:
            flood_score += 15
        
        if precip_mm > 20:
            flood_score += 15
        
        flood_score = min(flood_score, 100)
        
        output = {
            'hazard_type': 'COASTAL_FLOOD',
            'detected': flood_score > 30,
            'flood_score': flood_score,
            'tide_status': tide_status,
            'risk_level': risk_level,
            'alert_level': alert_level,
            'tide_height_m': tide_height_mt,
            'storm_surge_risk': storm_surge_risk,
            'onshore_winds': onshore_winds,
            'heavy_rainfall': precip_mm > 20,
            'timestamp': datetime.now().isoformat()
        }
        
        return output

# ============================================================================
# 6. REAL-TIME DATA COLLECTORS - FIXED
# ============================================================================

class RealTimeDataCollector:
    """
    Fetches real-time data from USGS and WeatherAPI
    FIXED: Better error handling and None value handling
    """
    
    def __init__(self, weather_api_key: str):
        self.weather_api_key = weather_api_key
        self.usgs_url = Config.USGS_API_URL
        
    def fetch_recent_earthquakes(self, min_magnitude: float = 5.5) -> List[Dict]:
        """
        Fetch recent earthquakes from USGS
        
        ** OUTPUT FOR FRONTEND **
        """
        try:
            response = requests.get(self.usgs_url, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            earthquakes = []
            for feature in data['features']:
                props = feature['properties']
                coords = feature['geometry']['coordinates']
                
                if props['mag'] >= min_magnitude:
                    earthquake = {
                        'id': feature['id'],
                        'magnitude': props['mag'],
                        'depth': coords[2],
                        'latitude': coords[1],
                        'longitude': coords[0],
                        'location': props['place'],
                        'time': datetime.fromtimestamp(props['time']/1000).isoformat(),
                        'tsunami_flag': props.get('tsunami', 0),
                        'location_type': self._classify_location_type(coords[1], coords[0]),
                        'url': props['url']
                    }
                    earthquakes.append(earthquake)
            
            return earthquakes
            
        except Exception as e:
            print(f"Error fetching earthquake data: {e}")
            return []
    
    def _classify_location_type(self, lat: float, lon: float) -> str:
        """Classify if earthquake is oceanic or land-based"""
        # Simplified classification
        if (-60 < lat < 60) and ((lon < -100) or (lon > 100)):
            return 'oceanic'
        if (-30 < lat < 30) and (40 < lon < 100):
            return 'oceanic'
        if (-60 < lat < 60) and (-80 < lon < 20):
            return 'oceanic'
        return 'land'
    
    def fetch_weather_for_location(self, location_id: str, location_data: Dict) -> Optional[Dict]:
        """
        Fetch weather and marine data for a specific location
        FIXED: Better None handling
        
        ** OUTPUT FOR FRONTEND **
        """
        try:
            lat = location_data['coords']['lat']
            lon = location_data['coords']['lon']
            
            marine_url = f"http://api.weatherapi.com/v1/marine.json"
            params = {
                'key': self.weather_api_key,
                'q': f"{lat},{lon}",
                'days': 1
            }
            
            response = requests.get(marine_url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            current = data.get('current', {})
            forecast = data.get('forecast', {}).get('forecastday', [])
            
            # Get marine data - handle missing forecast data
            if forecast and len(forecast) > 0:
                hours = forecast[0].get('hour', [])
                marine = hours[0] if hours else {}
            else:
                marine = {}
            
            weather_data = {
                'location_id': location_id,
                'location_name': location_data['name'],
                'timestamp': datetime.now().isoformat(),
                'coordinates': location_data['coords'],
                
                # Current weather
                'temperature_c': safe_get_value(current, 'temp_c', 25),
                'feels_like_c': safe_get_value(current, 'feelslike_c', 25),
                'condition': safe_get_value(current.get('condition', {}), 'text', 'Unknown'),
                'humidity': safe_get_value(current, 'humidity', 70),
                'cloud': safe_get_value(current, 'cloud', 50),
                'uv': safe_get_value(current, 'uv', 5),
                'visibility_km': safe_get_value(current, 'vis_km', 10),
                
                # Wind data
                'wind_kph': safe_get_value(current, 'wind_kph', 0),
                'wind_degree': safe_get_value(current, 'wind_degree', 0),
                'wind_dir': safe_get_value(current, 'wind_dir', 'N'),
                'gust_kph': safe_get_value(current, 'gust_kph', 0),
                
                # Pressure and precipitation
                'pressure_mb': safe_get_value(current, 'pressure_mb', 1013),
                'precip_mm': safe_get_value(current, 'precip_mm', 0),
                'will_it_rain': safe_get_value(marine, 'will_it_rain', 0),
                
                # Marine data - with defaults if not available
                'sig_ht_mt': safe_get_value(marine, 'sig_ht_mt', 1.0),
                'swell_ht_mt': safe_get_value(marine, 'swell_ht_mt', 1.0),
                'swell_period_secs': safe_get_value(marine, 'swell_period_secs', 10),
                'swell_dir': safe_get_value(marine, 'swell_dir', 'N'),
                'swell_dir_16_point': safe_get_value(marine, 'swell_dir_16_point', 'N'),
                'water_temp_c': safe_get_value(marine, 'water_temp_c', 25),
                
                # Tide information
                'tide_time': safe_get_value(marine, 'tide_time', ''),
                'tide_height_mt': safe_get_value(marine, 'tide_height_mt', 2.0),
                'tide_type': safe_get_value(marine, 'tide_type', 'normal'),
                
                # Astronomy
                'is_day': safe_get_value(current, 'is_day', 1),
                'last_updated': safe_get_value(current, 'last_updated', datetime.now().isoformat())
            }
            
            return weather_data
            
        except requests.exceptions.HTTPError as e:
            print(f"HTTP Error fetching weather for {location_id}: {e}")
            if hasattr(e, 'response') and e.response is not None:
                print(f"Response: {e.response.text}")
            return None
        except Exception as e:
            print(f"Error fetching weather for {location_id}: {e}")
            return None

# ============================================================================
# 7. INTEGRATED HAZARD DETECTION SYSTEM
# ============================================================================

class IntegratedHazardDetector:
    """
    Main class that combines all hazard detectors
    """
    
    def __init__(self):
        self.tsunami_model = TsunamiDetectionModel()
        self.feature_engineer = HazardFeatureEngineer()
        
    def train_tsunami_model(self, n_samples=2000):
        """Train tsunami detection model"""
        print("Generating training data...")
        df = generate_training_data(n_samples)
        print(f"Generated {len(df)} samples")
        
        print("\nTraining tsunami detection model...")
        metrics = self.tsunami_model.train(df)
        
        return metrics
    
    def detect_all_hazards(self, earthquake_data, weather_data):
        """
        Run all hazard detections
        
        ** THIS ENTIRE OUTPUT GOES TO YOUR FRONTEND **
        """
        results = {
            'timestamp': datetime.now().isoformat(),
            'earthquake_data': earthquake_data,
            'weather_summary': {
                'pressure_mb': safe_get_value(weather_data, 'pressure_mb', None),
                'wind_kph': safe_get_value(weather_data, 'wind_kph', None),
                'wave_height_m': safe_get_value(weather_data, 'sig_ht_mt', None),
                'weather_score': self.feature_engineer.calculate_weather_score(weather_data)
            },
            'hazards': {}
        }
        
        # 1. Tsunami Detection
        tsunami_features = self.feature_engineer.create_tsunami_features(
            earthquake_data, weather_data
        )
        tsunami_features['weather_score'] = results['weather_summary']['weather_score']
        tsunami_result = self.tsunami_model.predict(tsunami_features)
        results['hazards']['tsunami'] = tsunami_result
        
        # 2. Cyclone Detection
        cyclone_result = CycloneDetector.detect(weather_data)
        results['hazards']['cyclone'] = cyclone_result
        
        # 3. High Wave Detection
        wave_result = HighWaveDetector.detect(weather_data)
        results['hazards']['high_waves'] = wave_result
        
        # 4. Flood Detection
        flood_result = FloodDetector.detect(weather_data)
        results['hazards']['coastal_flood'] = flood_result
        
        # 5. Overall Risk Assessment
        max_alert_level = max([
            tsunami_result['alert_level'],
            cyclone_result['alert_level'],
            wave_result['alert_level'],
            flood_result['alert_level']
        ])
        
        active_hazards = []
        if tsunami_result['probability'] > 0.3:
            active_hazards.append('TSUNAMI')
        if cyclone_result['detected']:
            active_hazards.append('CYCLONE')
        if wave_result['detected']:
            active_hazards.append('HIGH_WAVES')
        if flood_result['detected']:
            active_hazards.append('COASTAL_FLOOD')
        
        results['overall_assessment'] = {
            'max_alert_level': max_alert_level,
            'active_hazards': active_hazards,
            'hazard_count': len(active_hazards),
            'requires_immediate_action': max_alert_level >= 4,
            'recommendations': self._generate_recommendations(results['hazards'])
        }
        
        return results
    
    def _generate_recommendations(self, hazards):
        """Generate action recommendations"""
        recommendations = []
        
        tsunami = hazards['tsunami']
        cyclone = hazards['cyclone']
        waves = hazards['high_waves']
        flood = hazards['coastal_flood']
        
        if tsunami['alert_level'] >= 4:
            recommendations.append("EVACUATE COASTAL AREAS IMMEDIATELY")
            recommendations.append("Move to higher ground (minimum 30m above sea level)")
            recommendations.append("Stay away from beaches and harbors")
        
        if cyclone['alert_level'] >= 4:
            recommendations.append("SEEK SHELTER IN STURDY BUILDINGS")
            recommendations.append("Secure loose objects outdoors")
            recommendations.append("Stock emergency supplies (food, water, medicine)")
        
        if waves['alert_level'] >= 3:
            recommendations.append("AVOID ALL BEACH ACTIVITIES")
            recommendations.append("Extreme rip current danger")
            recommendations.append("All boats stay in harbor")
        
        if flood['alert_level'] >= 3:
            recommendations.append("AVOID LOW-LYING COASTAL AREAS")
            recommendations.append("Prepare for storm surge")
            recommendations.append("Monitor tide times closely")
        
        if not recommendations:
            recommendations.append("Continue monitoring conditions")
            recommendations.append("Follow local authority guidance")
        
        return recommendations
    
    def save_all_models(self, prefix='hazard_system'):
        """Save trained models"""
        self.tsunami_model.save_model(f'{prefix}_tsunami_model.pkl')

# ============================================================================
# 8. MULTI-LOCATION HAZARD MONITOR
# ============================================================================

class MultiLocationHazardMonitor:
    """
    Monitors hazards across multiple locations simultaneously
    """
    
    def __init__(self, weather_api_key: str, hazard_detector):
        self.data_collector = RealTimeDataCollector(weather_api_key)
        self.hazard_detector = hazard_detector
        self.locations_db = CoastalLocationsDB()
        self.location_history = defaultdict(list)
        
    def calculate_distance(self, eq_lat: float, eq_lon: float, 
                          loc_lat: float, loc_lon: float) -> float:
        """Calculate distance between earthquake and location (km)"""
        from math import radians, cos, sin, asin, sqrt
        
        eq_lat, eq_lon, loc_lat, loc_lon = map(radians, [eq_lat, eq_lon, loc_lat, loc_lon])
        
        dlat = loc_lat - eq_lat
        dlon = loc_lon - eq_lon
        a = sin(dlat/2)**2 + cos(eq_lat) * cos(loc_lat) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a))
        
        return c * 6371  # Earth radius in km
    
    def monitor_single_location(self, location_id: str) -> Dict:
        """
        Monitor hazards for a single location
        
        ** OUTPUT FOR FRONTEND - INDIVIDUAL LOCATION CARD **
        """
        location_info = self.locations_db.get_location(location_id)
        if not location_info:
            return {'error': f'Location {location_id} not found'}
        
        weather_data = self.data_collector.fetch_weather_for_location(location_id, location_info)
        if not weather_data:
            return {'error': f'Failed to fetch weather data for {location_id}'}
        
        recent_earthquakes = self.data_collector.fetch_recent_earthquakes(min_magnitude=5.5)
        nearby_earthquakes = []
        
        for eq in recent_earthquakes:
            distance = self.calculate_distance(
                eq['latitude'], eq['longitude'],
                location_info['coords']['lat'], location_info['coords']['lon']
            )
            if distance <= 2000:
                eq['distance_to_location_km'] = round(distance, 2)
                nearby_earthquakes.append(eq)
        
        hazard_results = {
            'location_id': location_id,
            'location_name': location_info['name'],
            'country': location_info['country'],
            'coordinates': location_info['coords'],
            'region': location_info['region'],
            'risk_profile': location_info['risk_profile'],
            'population': location_info['population'],
            'timestamp': datetime.now().isoformat(),
            'weather_data': weather_data,
            'nearby_earthquakes': nearby_earthquakes,
            'hazard_detections': {}
        }
        
        if nearby_earthquakes:
            closest_eq = min(nearby_earthquakes, key=lambda x: x['distance_to_location_km'])
            
            earthquake_data = {
                'magnitude': closest_eq['magnitude'],
                'depth': closest_eq['depth'],
                'latitude': closest_eq['latitude'],
                'longitude': closest_eq['longitude'],
                'location_type': closest_eq['location_type']
            }
            
            detection_results = self.hazard_detector.detect_all_hazards(
                earthquake_data, weather_data
            )
            
            hazard_results['hazard_detections'] = detection_results
            hazard_results['closest_earthquake'] = closest_eq
            
        else:
            weather_score = HazardFeatureEngineer.calculate_weather_score(weather_data)
            cyclone_result = CycloneDetector.detect(weather_data)
            wave_result = HighWaveDetector.detect(weather_data)
            flood_result = FloodDetector.detect(weather_data)
            
            hazard_results['hazard_detections'] = {
                'weather_summary': {'weather_score': weather_score},
                'hazards': {
                    'tsunami': {'probability': 0, 'risk_level': 'NONE', 'alert_level': 1},
                    'cyclone': cyclone_result,
                    'high_waves': wave_result,
                    'coastal_flood': flood_result
                }
            }
        
        self.location_history[location_id].append(hazard_results)
        
        return hazard_results
    
    def monitor_all_locations(self, location_ids: Optional[List[str]] = None) -> Dict:
        """
        Monitor hazards for all locations
        
        ** THIS IS YOUR MAIN FRONTEND DATA SOURCE **
        """
        if location_ids is None:
            locations = self.locations_db.get_all_locations()
            location_ids = list(locations.keys())
        
        results = {
            'timestamp': datetime.now().isoformat(),
            'total_locations': len(location_ids),
            'locations': {},
            'global_earthquakes': [],
            'summary': {
                'critical_alerts': 0,
                'high_alerts': 0,
                'medium_alerts': 0,
                'locations_at_risk': []
            }
        }
        
        earthquakes = self.data_collector.fetch_recent_earthquakes(min_magnitude=5.5)
        results['global_earthquakes'] = earthquakes
        
        for loc_id in location_ids:
            print(f"Monitoring {loc_id}...")
            loc_results = self.monitor_single_location(loc_id)
            results['locations'][loc_id] = loc_results
            
            if 'hazard_detections' in loc_results and 'hazards' in loc_results['hazard_detections']:
                hazards = loc_results['hazard_detections']['hazards']
                
                max_alert = max([
                    hazards.get('tsunami', {}).get('alert_level', 1),
                    hazards.get('cyclone', {}).get('alert_level', 1),
                    hazards.get('high_waves', {}).get('alert_level', 1),
                    hazards.get('coastal_flood', {}).get('alert_level', 1)
                ])
                
                if max_alert >= 5:
                    results['summary']['critical_alerts'] += 1
                    results['summary']['locations_at_risk'].append({
                        'location_id': loc_id,
                        'location_name': loc_results['location_name'],
                        'alert_level': max_alert
                    })
                elif max_alert >= 4:
                    results['summary']['high_alerts'] += 1
                    results['summary']['locations_at_risk'].append({
                        'location_id': loc_id,
                        'location_name': loc_results['location_name'],
                        'alert_level': max_alert
                    })
                elif max_alert >= 3:
                    results['summary']['medium_alerts'] += 1
            
            time.sleep(1)  # Rate limiting
        
        return results

# ============================================================================
# 9. CONTINUOUS MONITORING SYSTEM
# ============================================================================

class ContinuousMonitoringSystem:
    """
    Background system that continuously monitors all locations
    """
    
    def __init__(self, weather_api_key: str, hazard_detector, 
                 update_interval_seconds: int = 300):
        self.monitor = MultiLocationHazardMonitor(weather_api_key, hazard_detector)
        self.update_interval = update_interval_seconds
        self.is_running = False
        self.latest_data = None
        self.monitoring_thread = None
        
    def start_monitoring(self, location_ids: Optional[List[str]] = None):
        """Start continuous monitoring in background thread"""
        if self.is_running:
            print("Monitoring already running!")
            return
        
        self.is_running = True
        self.monitoring_thread = threading.Thread(
            target=self._monitoring_loop,
            args=(location_ids,),
            daemon=True
        )
        self.monitoring_thread.start()
        print(f"✅ Continuous monitoring started (updates every {self.update_interval}s)")
    
    def stop_monitoring(self):
        """Stop continuous monitoring"""
        self.is_running = False
        if self.monitoring_thread:
            self.monitoring_thread.join(timeout=10)
        print("❌ Monitoring stopped")
    
    def _monitoring_loop(self, location_ids):
        """Internal monitoring loop"""
        while self.is_running:
            try:
                print(f"\n[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Fetching updates...")
                
                results = self.monitor.monitor_all_locations(location_ids)
                self.latest_data = results
                
                if results['summary']['critical_alerts'] > 0:
                    print("🚨 CRITICAL ALERTS DETECTED!")
                    for loc in results['summary']['locations_at_risk']:
                        if loc['alert_level'] >= 5:
                            print(f"   - {loc['location_name']}: Alert Level {loc['alert_level']}")
                
                print(f"✅ Update complete. Next update in {self.update_interval}s")
                
            except Exception as e:
                print(f"❌ Error in monitoring loop: {e}")
            
            time.sleep(self.update_interval)
    
    def get_latest_data(self) -> Optional[Dict]:
        """
        Get the latest monitoring data
        
        ** CALL THIS FROM YOUR FASTAPI ENDPOINT **
        """
        return self.latest_data

# ============================================================================
# 10. MAIN TESTING FUNCTION
# ============================================================================

def main():
    """
    Main testing function - Run this to test everything
    """
    print("=" * 70)
    print("MULTI-LOCATION COASTAL HAZARD DETECTION SYSTEM")
    print("=" * 70)
    
    # Check API key
    if Config.WEATHER_API_KEY == "your_weatherapi_key_here":
        print("\n❌ ERROR: WeatherAPI key not configured!")
        print("\nPlease follow these steps:")
        print("1. Go to: https://www.weatherapi.com/")
        print("2. Sign up for a FREE account")
        print("3. Copy your API key")
        print("4. Replace 'your_weatherapi_key_here' with your actual key")
        print("   in the Config class at the top of this file")
        print("\nExample:")
        print("   WEATHER_API_KEY = 'a1b2c3d4e5f6g7h8i9j0'")
        return
    
    # Initialize system
    print("\n1️⃣  Initializing system...")
    detector = IntegratedHazardDetector()
    
    # Train tsunami model
    print("\n2️⃣  Training tsunami detection model...")
    detector.train_tsunami_model(n_samples=2000)
    detector.save_all_models()
    
    # Initialize multi-location monitor
    print("\n3️⃣  Initializing multi-location monitor...")
    multi_monitor = MultiLocationHazardMonitor(Config.WEATHER_API_KEY, detector)
    
    # Test with high-risk locations only
    print("\n4️⃣  Monitoring high-risk locations...")
    high_risk_locs = CoastalLocationsDB.get_high_risk_locations()
    print(f"Found {len(high_risk_locs)} high-risk locations")
    
    # Monitor first 3 locations for demo
    test_locations = list(high_risk_locs.keys())[:3]
    print(f"Testing with: {', '.join(test_locations)}")
    
    results = multi_monitor.monitor_all_locations(test_locations)
    
    # Display results
    print("\n" + "=" * 70)
    print("📊 MONITORING RESULTS")
    print("=" * 70)
    print(f"Timestamp: {results['timestamp']}")
    print(f"Locations Monitored: {results['total_locations']}")
    print(f"Recent Earthquakes: {len(results['global_earthquakes'])}")
    print(f"Critical Alerts: {results['summary']['critical_alerts']}")
    print(f"High Alerts: {results['summary']['high_alerts']}")
    print(f"Medium Alerts: {results['summary']['medium_alerts']}")
    
    if results['summary']['locations_at_risk']:
        print(f"\n🚨 Locations at Risk:")
        for loc in results['summary']['locations_at_risk']:
            print(f"   - {loc['location_name']}: Alert Level {loc['alert_level']}")
    
    # Show detailed results for one location
    print("\n" + "=" * 70)
    print("📍 DETAILED RESULTS - First Location")
    print("=" * 70)
    
    first_loc_id = test_locations[0]
    first_loc = results['locations'][first_loc_id]
    
    print(f"Location: {first_loc['location_name']}, {first_loc['country']}")
    print(f"Region: {first_loc['region']}")
    print(f"Risk Profile: {first_loc['risk_profile']}")
    print(f"Population: {first_loc['population']:,}")
    
    if 'weather_data' in first_loc:
        weather = first_loc['weather_data']
        print(f"\n🌤  Current Weather:")
        print(f"   Temperature: {weather.get('temperature_c')}°C")
        print(f"   Wind: {weather.get('wind_kph')} km/h {weather.get('wind_dir')}")
        print(f"   Pressure: {weather.get('pressure_mb')} mb")
        print(f"   Wave Height: {weather.get('sig_ht_mt')} m")
    
    if 'hazard_detections' in first_loc and 'hazards' in first_loc['hazard_detections']:
        hazards = first_loc['hazard_detections']['hazards']
        print(f"\n⚠️  Hazard Status:")
        print(f"   Tsunami: {hazards.get('tsunami', {}).get('risk_level', 'N/A')}")
        print(f"   Cyclone: {hazards.get('cyclone', {}).get('category', 'N/A')}")
        print(f"   High Waves: {hazards.get('high_waves', {}).get('wave_condition', 'N/A')}")
        print(f"   Flood: {hazards.get('coastal_flood', {}).get('tide_status', 'N/A')}")
    
    # Save results
    print("\n" + "=" * 70)
    print("💾 SAVING RESULTS")
    print("=" * 70)
    
    with open('monitoring_results.json', 'w') as f:
        json.dump(results, f, indent=2, default=str)
    print("✅ Results saved to 'monitoring_results.json'")
    
    print("\n" + "=" * 70)
    print("✅ TESTING COMPLETE!")
    print("=" * 70)
    
    print("\n📤 NEXT STEPS FOR FRONTEND INTEGRATION:")
    print("""
    1. FastAPI Endpoints to create:
       - GET /api/locations → List all locations
       - GET /api/locations/{id} → Single location data
       - GET /api/locations/all/monitor → Monitor all locations
       - WebSocket /ws/monitoring → Real-time updates
    
    2. Frontend Components to build:
       - Map view with location markers
       - Location detail cards
       - Alert notification system
       - Real-time data dashboard
    
    3. Data structure is ready in 'monitoring_results.json'
       Use this as reference for your frontend
    """)
    
    return results

# ============================================================================
# RUN THE SYSTEM
# ============================================================================

if __name__ == "__main__":
    results = main()

MULTI-LOCATION COASTAL HAZARD DETECTION SYSTEM

1️⃣  Initializing system...

2️⃣  Training tsunami detection model...
Generating training data...
Generated 2000 samples

Training tsunami detection model...
=== TSUNAMI MODEL TRAINING RESULTS ===
Accuracy: 0.9825

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       287
           1       0.97      0.96      0.97       113

    accuracy                           0.98       400
   macro avg       0.98      0.98      0.98       400
weighted avg       0.98      0.98      0.98       400


Top 5 Important Features:
  is_oceanic: 0.3201
  mag_depth_ratio: 0.3038
  magnitude: 0.2532
  energy_release: 0.0627
  depth: 0.0352
Model saved to hazard_system_tsunami_model.pkl

3️⃣  Initializing multi-location monitor...

4️⃣  Monitoring high-risk locations...
Found 10 high-risk locations
Testing with: mumbai, chennai, visakhapatnam
Monitoring mumbai...
Monitoring chennai...
Mon